In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from patsy import dmatrices
import sklearn
import seaborn as sns

walmart = pd.read_csv("Walmart_Store_sales.csv")
walmart.head()

walmart_group = walmart.groupby('Store')['Weekly_Sales'].sum()
print("Store Number {} has maximum Sales. Sum of Total Sales
{}".format(walmart_group.idxmax(),walmart_group.max()))

walmart_std = walmart.groupby('Store').agg({'Weekly_Sales':'std'})
print("Store Number {} has maximum Standard Deviation. STD {}".format(walmart_std['Weekly_Sales'].idxmax(),walmart_std['Weekly_Sales'].max()))

walmart2012 = walmart[(pd.to_datetime(walmart['Date']) >= pd.to_datetime('07-01-2012')) & (pd.to_datetime(walmart['Date']) <= pd.to_datetime('09-30-2012'))]
growth = walmart2012.groupby(['Store'])['Weekly_Sales'].sum()
print("Store Number {} has Good Quartely Growth in Q3'2012 {}".format(growth.idxmax(),growth.max()))

holiday = walmart[walmart['Holiday_Flag'] == 1]

nonholiday = walmart[walmart['Holiday_Flag'] == 0]

superBowl = holiday[(pd.to_datetime(holiday['Date']) == pd.to_datetime('12-02-2010')) |(pd.to_datetime(holiday['Date']) == pd.to_datetime('11-02-2011'))|(pd.to_datetime(holiday['Date']) == pd.to_datetime('10-02-2012'))|(pd.to_datetime(holiday['Date']) == pd.to_datetime('08-02-2013'))]

labourDay = holiday[(pd.to_datetime(holiday['Date']) == pd.to_datetime('10-09-2010')) |(pd.to_datetime(holiday['Date']) == pd.to_datetime('09-09-2011'))|(pd.to_datetime(holiday['Date']) == pd.to_datetime('07-09-2012'))|(pd.to_datetime(holiday['Date']) == pd.to_datetime('06-09-2013'))]

thanksgiving = holiday[(pd.to_datetime(holiday['Date']) == pd.to_datetime('26-11-2010')) |(pd.to_datetime(holiday['Date']) == pd.to_datetime('25-11-2011'))|(pd.to_datetime(holiday['Date']) == pd.to_datetime('23-11-2012'))|(pd.to_datetime(holiday['Date']) == pd.to_datetime('29-11-2013'))]

christmas = holiday[(pd.to_datetime(holiday['Date']) == pd.to_datetime('31-12-2010')) |(pd.to_datetime(holiday['Date']) == pd.to_datetime('30-12-2011'))|(pd.to_datetime(holiday['Date']) == pd.to_datetime('28-12-2012'))|(pd.to_datetime(holiday['Date']) == pd.to_datetime('27-12-2013'))]

nonholiday_mean = nonholiday.groupby(['Date']).agg({'Weekly_Sales':'mean'}).reset_index()
holiday_sum = holiday.groupby(['Date']).agg({'Weekly_Sales':'sum'}).reset_index()

print("Super Bowl Day Sale",superBowl['Weekly_Sales'].sum())
print("Labour Day Sale",labourDay['Weekly_Sales'].sum())
print("Thanksgiving Day Sale",thanksgiving['Weekly_Sales'].sum())
print("Christmas Day Sale",christmas['Weekly_Sales'].sum())

x = walmart[walmart['Store'] ==1][['Store','Date']]
date = walmart[walmart['Store'] ==1][['Date']]
date.index +=1
x.Date = date.index
x.head()

y = walmart[walmart['Store'] ==1]['Weekly_Sales']
y.head()

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=1)

from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(x_train,y_train)
linear = walmart[walmart['Store'] ==1][['Store','CPI','Unemployment','Fuel_Price'
]]
linear.head()

cpi = walmart[walmart['Store'] ==1]['CPI'].astype('int64')
unemployment = walmart[walmart['Store'] ==1]['Unemployment'].astype('int64')

from sklearn.model_selection import train_test_split
x_train_cpi,x_test_cpi,y_train_cpi,y_test_cpi = train_test_split(linear,cpi,random_state=1)
x_train_unemp,x_test_unemp,y_train_unemp,y_test_unemp = train_test_split(linear,unemployment,random_state=1)
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(max_iter=10000)
logreg.fit(x_train_cpi,y_train_cpi)
y_pred = logreg.predict(x_test_cpi)
logreg.fit(x_train_unemp,y_train_unemp)
y_pred_unemp = logreg.predict(x_test_unemp)

from sklearn import metrics
print(metrics.accuracy_score(y_test_cpi,y_pred))
print(metrics.accuracy_score(y_test_unemp,y_pred_unemp))

print('cpi actual :', y_test_cpi.values[0:30])
print('cpi Predicted :', y_pred[0:30])
print('actual Unemployment :', y_test_unemp.values[0:30])
print('Predicted Unemployment :', y_pred_unemp[0:30])

walmart['Day'] = pd.to_datetime(walmart['Date']).dt.day_name()
walmart.head()
